In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.preprocessing import preprocess_input_df

/Users/lorenh/Documents/DSaPP/.aequitas/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


SyntaxError: invalid syntax (bias.py, line 123)

In [ ]:
df = pd.read_csv("../../../examples/data/compas_for_aequitas.csv")
df.head()

In [ ]:
df.shape

## Pre-Aequitas: Exploring the COMPAS Dataset

__Risk assessment by race__

COMPAS produces a risk score that predicts a person's likelihood of commiting a crime in the next two years. The output is a score between 1 and 10 that maps to low, medium or high. For Aequitas, we collapse this to a binary prediction. A score of 0 indicates a prediction of "low" risk according to COMPAS, while a 1 indicates "high" or "medium" risk.

This categorization is based on ProPublica's interpretation of Northpointe's practioner guide:

    "According to Northpointe’s practitioners guide, COMPAS “scores in the medium and high range garner more interest from supervision agencies than low scores, as a low score would suggest there is little risk of general recidivism,” so we considered scores any higher than “low” to indicate a risk of recidivism."

In the bar charts below, we see a large difference in how these scores are distributed by race, with a majority of white and Hispanic people predicted as low risk (score = 0) and a majority of black people predicted high and medium risk (score = 1). We also see that while the majority of people in age categories over 25 are predicted as low risk (score = 0), the majority of people below 25 are predicted as high and medium risk (score = 1).

### Data Formatting

Data for this example was preprocessed for compatibility with Aequitas. **The Aequitas tool always requires a `score` column and requires a binary `label_value` column for supervised metrics**, (i.e., False Discovery Rate, False Positive Rate, False Omission Rate, and False Negative Rate).

Preprocessing includes but is not limited to checking for mandatory `score` and `label_value` columns as well as at least one column representing attributes specific to the data set. See [documentation](../input_data.html) for more information about input data.

Note that while `entity_id` is not necessary for this example, Aequitas recognizes `entity_id` as a reserve column name and will not recognize it as an attribute column.

[Back to Top](#top_cell)
<a id='existing_biases'></a>
## What biases exist in my model?
### Aequitas Group() Class

<a id='xtab'></a>
### What is the distribution of groups, predicted scores, and labels across my dataset?

Aequitas's `Group()` class enables researchers to evaluate biases across all subgroups in their dataset by assembling a confusion matrix of each subgroup, calculating commonly used metrics such as false positive rate and false omission rate, as well as counts by group and group prevelance among the sample population. 

The **`get_crosstabs()`** command tabulates a confusion matrix for each subgroup and calculates commonly used metrics such as false positive rate and false omission rate. It also provides counts by group and group prevelances.

#### Group Counts Calculated:

| Count Type | Column Name |
| --- | --- |
| False Positive Count | 'fp' |
| False Negative Count | 'fn' |
| True Negative Count | 'tn' |
| True Positive Count | 'tp' |
| Predicted Positive Count | 'pp' |
| Predicted Negative Count | 'pn' |
| Count of Negative Labels in Group | 'group_label_neg' |
| Count of Positive Labels in Group | 'group_label_pos' | 
| Group Size | 'group_size'|
| Total Entities | 'total_entities' |

#### Absolute Metrics Calcuated:

| Metric | Column Name |
| --- | --- |
| True Positive Rate | 'tpr' |
| True Negative Rate | 'tnr' |
| False Omission Rate | 'for' |
| False Discovery Rate | 'fdr' |
| False Positive Rate | 'fpr' |
| False Negative Rate | 'fnr' |
| Negative Predictive Value | 'npv' |
| Precision | 'precision' |
| Predicted Positive Ratio$_k$ | 'ppr' |
| Predicted Positive Ratio$_g$ | 'pprev' |
| Group Prevalence | 'prev' |


**Note**: The **`get_crosstabs()`** method expects a dataframe with predefined columns `score`, and `label_value` and treats other columns (with a few exceptions) as attributes against which to test for disparities. In this cases we include `race`, `sex` and `age_cat`. 

In [ ]:
g = Group()
xtab, _, score_thresholds_dict = g.get_crosstabs(df)

In [ ]:
preprocessed_df, _ = preprocess_input_df(df, required_cols=['score', 'label_value'])

In [ ]:
preprocessed_df.head()

In [ ]:
df.head()

### Next Step: Incorporate into Bias() class! 

Will likely want to drop the binary columns
Consider: how to keep detail from original df but calculate significance based on the group ID'd as ref_group in bias df?

[Back to Top](#top_cell)
<a id='xtab_metrics'></a>
### What are bias metrics across groups?

Once you have run the `Group()` class, you'll have a dataframe of the group counts and group value bias metrics.

The `Group()` class has a **`list_absolute_metrics()`** method, which you can use for faster slicing to view just  counts or bias metrics.

In [ ]:
absolute_metrics = g.list_absolute_metrics(xtab)

#### View calculated counts across sample population groups

In [ ]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

#### View calculated absolute metrics for each sample population group

In [ ]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2)

In [ ]:
b = Bias()

In [ ]:
rd = b.get_disparity_predefined_groups(xtab, ref_groups_dict={'sex': 'Female' , 'race': 'Hispanic', 'age_cat': '25 - 45'})

In [ ]:
xtab.loc[xtab.group_size.idxmax(), 'attribute_value']

In [ ]:
rd

In [ ]:
bodf = b.get_disparity_major_group(xtab)

In [ ]:
default_key_columns = ['model_id', 'score_threshold', 'attribute_name']

In [ ]:
df_min_idx = bodf.loc[bodf.groupby(default_key_columns)['fpr'].idxmin()]
df_min_idx

In [ ]:
df_min_idx.loc[df_min_idx['attribute_name'] == 'race']

In [ ]:
b.get_disparity_min_metric(xtab)